# Importing needed libraries

In [1]:
import os
import re
from datetime import datetime
import pandas as pd
import csv
from tqdm import tqdm
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer as ps
from nltk.tokenize import RegexpTokenizer
import json
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import jaccard_score
import collections
import heapq

# 1.1 Get the list of animes

With the for loop I get the url of all the pages (50 per page except the last one which has less,383 pag). We 'send' a request to the page, to collect the ursl. We read the html and take the href attributes of the tag. The command tqdm is useful because it shows the progress status.

In [ ]:
urls=[]
for i in tqdm(range(0,20000,50)):
  url='https://myanimelist.net/topanime.php?limit='+str(i)
  
  soup= BeautifulSoup(requests.get(url).text,'html.parser')

  for tag in soup.find_all('tr'):
    links=tag.find_all('a')
    for l in links:
      if type(l.get('id'))== str and len(l.contents[0]) >1:
        urls.append(l.get('href'))



We need to save it as text

In [ ]:
with open('urls.txt', 'w', encoding='utf-8') as f: #open the file con write, chiude da solo in automatico, lo trovo in file sample data
    for line in urls:
        f.write(line)
        f.write('\n')

1.2 Crawl animes

Create the folders. Put all of them under "pages". Each folder has a name that refers to the number of the page from which the links it contains come from.

In [ ]:
for page in tqdm(range(1, 384)):
    folder = "page"+str(page)
    path = "/content/drive/MyDrive/Anime_folder"+folder
    os.mkdir(path)

Try for the first 150:ok. Try to collect them in group of 3.

In [ ]:
for page in tqdm(range(4, 5)): 
   
  folder = "/content/drive/MyDrive/Anime_folderpage"+str(page+1)
  update_page = 50*page
  for i in range(0,50): 
    time.sleep(3) 
    url = f'{urls[update_page+i]}'
    response = requests.get(url)   
    filename = r""+folder+"/anime_"+str(update_page+i+1)+".html"
    with open(filename,'w', encoding='utf-8') as f:
      f.write(response.text)

**1.3**
Parsing downloaded pages.
Here I created a function to extract each of the features that we need.



In [ ]:
def animeTitle(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f, 'html.parser')
    animeTitle=soup.find("h1", attrs = {"class": "title-name h1_bold_none"}).string
    return(animeTitle)



In [ ]:
animeTitle('/content/to8760/article_4016.html')

In [ ]:
def animeScore(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f)
    for i in range(10):
      score=soup.find_all('div' ,attrs = {"class": "score-label score-"+str(i)})
      for j in score:
        return float(j.text)

In [ ]:
animeScore('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def animeRank(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f,'html.parser')
    b= soup.find('span',{'class':'numbers ranked'})
    rank=int(b.find('strong').contents[0].strip()[1:])  
    return(rank)

In [ ]:
animeRank('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')



In [ ]:
def animeNumberofepisode(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=soup.find(text=re.compile('Episodes:')).parent.parent.text.split()[-1]      
    return(A)

In [ ]:
animeNumberofepisode('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')



In [ ]:
def animePopularity(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f,'html.parser')
    rank=int(str(soup.find(class_="numbers popularity").text).split('#')[-1])  
    return(rank)

In [ ]:
animePopularity('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def animeDescription(html_string): #I think that this function works well.
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    A=soup.find(itemprop="description")
    return A.get_text()

In [ ]:
animeDescription('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def animeUsers(html_string): 
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    A=soup.find(itemprop="ratingCount")
    return int(A.get_text())

In [ ]:
animeUsers('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def animeType(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A=soup.find(text=re.compile('Type:')).parent.parent.text.split()[-1]      
    return(A)

In [ ]:
animeType('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def fai_date(a: list):
    
    string = ''
    for i in a:
        string += i
        if i != a[-1]:
            string += ' '
    string = string.replace(',', '')
    date = datetime.strptime(string, '%b %d %Y')
    return date

In [ ]:
def animeRelDate(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= fai_date(soup.find(text=re.compile('Aired:'), class_="dark_text").parent.text.split()[1:4]  ).date()   
    return(A)

In [ ]:
animeRelDate('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def animeEndDate(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= fai_date(soup.find(text=re.compile('Aired:'), class_="dark_text").parent.text.split()[5:8]   ).date() 
    return(A)

In [ ]:
print(animeEndDate('/content/drive/MyDrive/Anime_folderpage1/anime_1.html'))

In [ ]:
def animeRelated(html_string):  
  animeRelated = []
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    related = soup.find_all("table", {"class":"anime_detail_related_anime"})
    for i in related:
      links = i.find_all('a')
      for link in links:        
          animeRelated.append(f'{link.contents[0]}')

  return (animeRelated)
     

In [ ]:
 animeRelated('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def animeCharacter(html_string):
  personaggi=[]
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= soup.find_all(class_="h3_characters_voice_actors")    
    for a in A:
      personaggi.append(a.text)
    return(personaggi) 

In [ ]:
animeCharacter('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def animeVoices(html_string):
  personaggi=[]
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    
    A= soup.find_all(class_="va-t ar pl4 pr4")   
    for a in A:
      personaggi.append(a.text)

      
    return(personaggi) 
   

In [ ]:
animeVoices('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
def animeStaff(html_string):
  with open(html_string, 'r') as f:
    soup = BeautifulSoup(f, 'html.parser')
    divs= soup.find_all('div',{'class':"detail-characters-list clearfix"})
    i=1
    staff=[]
    for td in divs[-1].find_all('td',{'class':'borderClass'}):
      if (i)==0:
        a,small= td.find_all(['small','a'])
        staff.append((a.contents[0],small.contents[0].split(',')))
      i= (i+1)%2
  return(staff)

In [ ]:
animeStaff('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

In [ ]:
 def animeNumMembers(html):
  with open(html,'r') as f:
    soup= BeautifulSoup(f,'html.parser')
    rank=  int(str(soup.find(class_="numbers members").text).split()[1].replace(',', ''))      
    return(rank)
 
 

In [ ]:
animeNumMembers('/content/drive/MyDrive/Anime_folderpage1/anime_1.html')

I need to crete a tsv.file.

In [ ]:
i = 1
col = ['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers', 'animeScore', \
           'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', \
           'animeVoices', 'animeStaff']
while i < 2600:
    
    path = '/content/drive/MyDrive/Anime_folderpage'+str(((i-1)//50)+1)+'/anime_'+str(i)+'.html'
    dic = {}
    
    
    dic[col[0]] = animeTitle(path)
    dic[col[1]] = animeType(path)
    dic[col[2]] = animeNumberofepisode(path)
    try:
      dic[col[3]] = animeRelDate(path)
    except Exception:
      dic[col[3]] = ""
    try:
      dic[col[4]] = animeEndDate(path)
    except Exception:
      dic[col[4]] = ""
    dic[col[5]]= animeNumMembers(path)
    dic[col[6]]= animeScore(path)
    dic[col[7]]= animeUsers(path)
    dic[col[8]]= animeRank(path)
    dic[col[9]]= animePopularity(path)
    dic[col[10]]= animeDescription(path)
    try:
      dic[col[11]]= animeRelated(path)
    except Exception:
      dic[col[11]] = ""
    dic[col[12]]= animeCharacter(path)
    dic[col[13]]= animeVoices(path)
    try:
      dic[col[14]]= animeStaff(path)
    except Exception:
       dic[col[14]] = ""
    with open('/content/drive/MyDrive/Anime_folderpage'+str(((i-1)//50)+1)+'/anime_'+str(i)+ '.tsv', 'w',encoding="utf-8") as page:
      i += 1
      for c in col:
        if c in dic:
           page.write(str(dic[c]) + '\t')
        else:
          page.write(' \t')
    
    

In [ ]:
with open('/content/drive/MyDrive/Anime_folderpage_name'+ '.tsv', 'w',encoding="utf-8") as page:
  for c in col:
    page.write(c+'\t')

# create dataframe

In [4]:
df = pd.DataFrame()
df = pd.DataFrame(columns = ['animeTitle', 'animeType', 'animeNumEpisode', 'releaseDate', 'endDate', 'animeNumMembers', 'animeScore', \
           'animeUsers', 'animeRank', 'animePopularity', 'animeDescription', 'animeRelated', 'animeCharacters', \
           'animeVoices', 'animeStaff'])

In [5]:
path = "/Users/mac/Desktop/I'm a data scientist b****/ADM/HMW_3/cosi/anime_"
def fillcsv(m,n):
    for elem in tqdm(range(m,n)):
        tsv_file = open(path+str(elem)+".tsv")
        read_tsv = csv.reader(tsv_file, delimiter="\t")
        cont = 0
        for anime in read_tsv:
            if cont == 1:
                anime = anime[0:15]
                df.loc[elem] = anime
            cont +=1
fillcsv(1,19128)


100%|██████████| 19127/19127 [02:58<00:00, 106.95it/s]


In [6]:

display(df)

,animeTitle,animeType,animeNumEpisode,releaseDate,endDate,animeNumMembers,animeScore,animeUsers,animeRank,animePopularity,animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff
1,Fullmetal Alchemist: Brotherhood,TV,64,2009-04-05 00:00:00,2010-07-04 00:00:00,2674644,9.16,1622384,1,3,After a horrific alchemy experiment goes wrong...,"['Fullmetal Alchemist', 'Fullmetal Alchemist: ...","['Elric, Edward', 'Elric, Alphonse', 'Mustang,...","['Park, Romi', 'Kugimiya, Rie', 'Miki, Shinich...","[['Cook Justin', ['Producer']], ['Yonai Norito..."
2,Gintama°,TV,51,2015-04-08 00:00:00,2016-03-30 00:00:00,483584,9.09,169476,2,337,"Gintoki, Shinpachi, and Kagura return as the f...","['Gintama', 'Gintama Movie 2: Kanketsu-hen - Y...","['Sakata, Gintoki', 'Kagura', 'Shimura, Shinpa...","['Sugita, Tomokazu', 'Kugimiya, Rie', 'Sakaguc...","[['Fujita Youichi', ['Director', 'Storyboard',..."
3,Shingeki no Kyojin Season 3 Part 2,TV,10,2019-04-29 00:00:00,2019-07-01 00:00:00,1594869,9.09,1087519,3,33,Seeking to restore humanity's diminishing hope...,"['Shingeki no Kyojin', 'Shingeki no Kyojin Sea...","['Levi', 'Ackerman, Mikasa', 'Yeager, Eren', '...","['Kamiya, Hiroshi', 'Ishikawa, Yui', 'Kaji, Yu...","[['Yabuta Shuuhei', ['Producer']], ['Wada Jouj..."
4,Steins;Gate,TV,24,2011-04-06 00:00:00,2011-09-14 00:00:00,2090123,9.09,1109700,4,11,The self-proclaimed mad scientist Rintarou Oka...,"['Steins;Gate', 'ChäoS;HEAd', 'Robotics;Notes'...","['Okabe, Rintarou', 'Makise, Kurisu', 'Shiina,...","['Miyano, Mamoru', 'Imai, Asami', 'Hanazawa, K...","[['Iwasa Gaku', ['Producer']], ['Yasuda Takesh..."
5,Fruits Basket: The Final,TV,13,2021-04-06 00:00:00,2021-06-29 00:00:00,274818,9.07,113310,5,651,"Hundreds of years ago, the Chinese Zodiac spir...","['Fruits Basket', 'Fruits Basket 2nd Season']","['Souma, Kyou', 'Honda, Tooru', 'Souma, Yuki',...","['Uchida, Yuuma', 'Iwami, Manaka', 'Shimazaki,...","[['Ibata Yoshihide', ['Director']], ['Aketagaw..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19123,Kaihouku: Chikan Harem,OVA,1,2007-03-25 00:00:00,,78,,,,18234,,['Tsuukin Kairaku: Chikan de Go!!'],[],[],
19124,Konbini Shoujo Z,OVA,,,,559,,,,12775,,,[],[],
19125,Korogashi Ryouta,OVA,3,1990-11-21 00:00:00,1991-11-21 00:00:00,310,,,,14511,Takao Ryota is a pervy but well-meaning biker-...,['Korogashi Ryouta'],[],[],"[['Ochiai Masamune', ['Director']]]"
19126,Kyonyuu Elf Oyako Saimin,OVA,2,2022-01-28 00:00:00,,128,,,,17357,,,"['Agraliel, Ephildis', 'Agraliel, Almia']",[],"[['Raika Ken', ['Director']]]"


In [157]:
df.loc[15009]

animeTitle                           Bikkuriman: Moen Zone no Himitsu
animeType                                                       Movie
animeNumEpisode                                                     1
releaseDate                                       1988-07-09 00:00:00
endDate                                                              
animeNumMembers                                                   263
animeScore                                                           
animeUsers                                                           
animeRank                                                       15009
animePopularity                                                 15027
animeDescription                                                     
animeRelated                                           ['Bikkuriman']
animeCharacters                                                    []
animeVoices                                                        []
animeStaff          

In [182]:
df.to_csv('Animeds.csv', index=False)

# 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the animes that match the query.

First, you must pre-process all the information collected for each anime by:

Removing stopwords
Removing punctuation
Stemming
Anything else you think it's needed
---

- Importing necessary libraries:

In [2]:
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Filippo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


- removing punctuation:

In [3]:
stringa = "Seeking to restore humanity's diminishing hope, the Survey Corps embark on a mission to retake Wall Maria, where the battle against the merciless Titans takes the stage once again.Returning to the tattered Shiganshina District that was once his home, Eren Yeager and the Corps find the town oddly unoccupied by Titans. Even after the outer gate is plugged, they strangely encounter no opposition. The mission progresses smoothly until Armin Arlert, highly suspicious of the enemy's absence, discovers distressing signs of a potential scheme against them. Shingeki no Kyojin Season 3 Part 2 follows Eren as he vows to take back everything that was once his. Alongside him, the Survey Corps strive—through countless sacrifices—to carve a path towards victory and uncover the secrets locked away in the Yeager family's basement"
stringa.translate(str.maketrans('', '', string.punctuation))

'Seeking to restore humanitys diminishing hope the Survey Corps embark on a mission to retake Wall Maria where the battle against the merciless Titans takes the stage once againReturning to the tattered Shiganshina District that was once his home Eren Yeager and the Corps find the town oddly unoccupied by Titans Even after the outer gate is plugged they strangely encounter no opposition The mission progresses smoothly until Armin Arlert highly suspicious of the enemys absence discovers distressing signs of a potential scheme against them Shingeki no Kyojin Season 3 Part 2 follows Eren as he vows to take back everything that was once his Alongside him the Survey Corps strive—through countless sacrifices—to carve a path towards victory and uncover the secrets locked away in the Yeager familys basement'

In [43]:
# INUTILE
vocab={}
n = df.shape[0]
wrdcount = 1
for i in tqdm(range(n)):
    desc = str(df.iloc[i]['animeDescription'])
    splitted = desc.translate(str.maketrans('', '', string.punctuation)).split(" ")
    #creating the dictionary for the indices
    for word in splitted:
        word = word.lower()
        if word not in stopwords.words('english'):
            if word not in vocab:
                vocab[word] = [wrdcount]
                wrdcount+=1

100%|████████████████████████████████████████████████████████████████████████████| 19127/19127 [04:06<00:00, 77.63it/s]


In [37]:

word_index = defaultdict(list)
#n will run through the length of the dataframe
n = df.shape[0]
for i in tqdm(range(0,n)):
    #fetching the anime descriptions
    desc = str(df.iloc[i]['animeDescription'])
    splitted = desc.translate(str.maketrans('', '', string.punctuation)).split(" ")
    #creating the dictionary for the indices
    for word in splitted:
        word = word.lower()
        if word not in stopwords.words('english'):
            if word not in word_index:
                word_index[word] = [[i]]
            else:
                if [i] not in word_index[word]:
                    word_index[word].append([i])



100%|████████████████████████████████████████████████████████████████████████████| 19127/19127 [04:53<00:00, 65.28it/s]


In [38]:
word_index['saiyan']

[[364], [400], [1468], [1962], [2015], [2294], [4339], [4673], [8969], [9229]]

In [24]:
def listoflists(list):
    flat_list = []
    for elem in list:
        flat_2 = []
        for single in elem:
            flat_2.append(single[0])
        flat_list.append(flat_2)
    return flat_list

def ReturnUrl(index):
    f = open("./urls.txt", encoding="utf8")
    stringa = ""
    for i, line in enumerate(f):
        if i == index:
            stringa = line
    return stringa

def search_engine1(query,dataset):
  query = query.split(" ")
  f = len(query)
  query_indices = []
  for index in query:
    query_indices.append(word_index[index])  
  query_indices = set.intersection(*map(set,listoflists(query_indices)))
  query_index = list(set(query_indices))
  return(query_index)

def displayresults(lista):
    disp_res = pd.DataFrame()
    disp_res = pd.DataFrame(columns = ['Anime Title','Anime Description','Anime URL'])
    res = []
    for i in lista:
        roba = []
        roba.append(df.iloc[i]['animeTitle'])
        roba.append(df.iloc[i]['animeDescription'])
        if i == 1468:
            roba.append(ReturnUrl(i-1))
        elif i >372:
            roba.append(ReturnUrl(i+1))
        else:
            roba.append(ReturnUrl(i))
        res.append(roba)
    index = 0
    for elem in res:
        disp_res.loc[index] = elem
        index += 1
    return(display(disp_res))

In [173]:
for i in range(1466,1470):
    print(df.iloc[i]['animeTitle'],ReturnUrl(i))

Donten ni Warau Gaiden: Shukumei, Soutou no Fuuma https://myanimelist.net/anime/905/Dragon_Ball_Z_Movie_12__Fukkatsu_no_Fusion_Gokuu_to_Vegeta

Dragon Ball Z Movie 12: Fukkatsu no Fusion!! Gokuu to Vegeta https://myanimelist.net/anime/986/Dragon_Ball_Z_Special_1__Tatta_Hitori_no_Saishuu_Kessen

Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen https://myanimelist.net/anime/36480/Drifters_OVA

Drifters (OVA) https://myanimelist.net/anime/31552/Durararax2_Ten__Onoroke_Chakapoko



In [179]:
df.iloc[6],ReturnUrl(1467)

(animeTitle                                     Hunter x Hunter (2011)
 animeType                                                          TV
 animeNumEpisode                                                   148
 releaseDate                                       2011-10-02 00:00:00
 endDate                                           2014-09-24 00:00:00
 animeNumMembers                                               2148879
 animeScore                                                       9.06
 animeUsers                                                    1252938
 animeRank                                                           7
 animePopularity                                                    10
 animeDescription    Hunter x Hunter is set in a world where Hunter...
 animeRelated        ['Hunter x Hunter', 'Hunter x Hunter: Original...
 animeCharacters     ['Zoldyck, Killua', 'Kurapika', 'Freecss, Gon'...
 animeVoices         ['Ise, Mariya', 'Sawashiro, Miyuki', 'Han, Meg...
 anime

In [181]:
query = str(input("Insert your anime features: "))
search_engine1(query,df),displayresults(search_engine1(query,df))

Insert your anime features:  saiyan race


,Anime Title,Anime Description,Anime URL
0,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...
1,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...
2,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n


([400, 1468, 364], None)

In [162]:
ReturnUrl(1467)

'https://myanimelist.net/anime/986/Dragon_Ball_Z_Special_1__Tatta_Hitori_no_Saishuu_Kessen\n'

In [ ]:
res

In [ ]:
with open('word_index.pkl', 'wb',) as f:
    pickle.dump(file=f,obj = word_index)

In [83]:
with open('vocabulary.pkl','wb') as f:
    pickle.dump(file=f,obj = vocab_2)

In [9]:
# loading dataframe
df = pd.read_csv('Animeds.csv')
word_index = pd.read_pickle('word_index.pkl')
vocab = pd.read_pickle('vocabulary.pkl')

## 2.2 Conjunctive query & Ranking score

In [95]:
from scipy.sparse import dia_matrix
import numpy as np
import scipy

In [13]:
vectorizer = TfidfVectorizer(stop_words='english')

In [15]:
n = df.shape[0]
descriptions = [str(df.iloc[index]['animeDescription']) for index in range(n)]
tf_idf_scores = vectorizer.fit_transform(iter(descriptions))
vocab = vectorizer.get_feature_names()

In [44]:
print(tf_idf_scores[:,0])
dicto = tf_idf_scores.T.todok()
it = dicto.keys()
next(iter(it))[0]

  (388, 0)	0.09378575745668703
  (1373, 0)	0.3424286505006758
  (3796, 0)	0.15218341701961385
  (3859, 0)	0.29293276550671893
  (4013, 0)	0.09096971424220497
  (4467, 0)	0.15836088560040337
  (5115, 0)	0.23562617888158202
  (7388, 0)	0.28559304142938974
  (9268, 0)	0.3642537322163859
  (10928, 0)	0.0877614230070796
  (17963, 0)	0.12473893111406961
  (18414, 0)	0.09824630606979153
  (18460, 0)	0.1473290026631456


1745

In [57]:
inverted_index_2 = defaultdict(list)
dicto.keys()
for tup in tqdm(iter(dicto.keys())):
    wrd = tup[0]
    if wrd not in inverted_index_2:           
        inverted_index_2[wrd] = [(tup[1], dicto[tup])]
    else:
        inverted_index_2[wrd].append(tuple((tup[1], dicto[tup])))

555619it [00:07, 75306.19it/s]


In [60]:
with open('inverted index 2.pkl', 'wb',) as f:
    pickle.dump(file=f,obj = inverted_index_2)

In [74]:
inverted_index_2[0]


[(388, 0.09378575745668703),
 (1373, 0.3424286505006758),
 (3796, 0.15218341701961385),
 (3859, 0.29293276550671893),
 (4013, 0.09096971424220497),
 (4467, 0.15836088560040337),
 (5115, 0.23562617888158202),
 (7388, 0.28559304142938974),
 (9268, 0.3642537322163859),
 (10928, 0.0877614230070796),
 (17963, 0.12473893111406961),
 (18414, 0.09824630606979153),
 (18460, 0.1473290026631456)]

In [ ]:
vocab_2 = {k: v for v, k in enumerate(vocab)}

In [90]:
vocab_2['male']

IndexError: invalid index to scalar variable.

In [122]:
tf_idf_query = vectorizer.transform(['saiyan', 'race', 'male'])
ty = tf_idf_query.todok()
np.array(list(ty.values()))

array([1., 1., 1.])

In [127]:
def search_engine2(dataset):
    query = str(input("Search these words: ") or 'saiyan race male')
    query = query.split(" ")
    tf_vect = TfidfVectorizer(stop_words='english',vocabulary=vocab_2) 
    tf_idf_query = tf_vect.fit_transform(query)
    tf_idf_query = tf_idf_query.todok()
    tf_idf_query = list(tf_idf_query.values())
    docs_to_check = {}
    wrd_count = 0
    for word in query:
        id_w = vocab_2[word]
        for elem in inverted_index_2[id_w]:
            if elem[0] not in docs_to_check:
                docs_to_check[elem[0]] = np.zeros(len(query))
            docs_to_check[elem[0]][wrd_count] = elem[1]
        wrd_count+=1
    for doc, vector in docs_to_check.items():      
        doc_sim = 1 - scipy.spatial.distance.cosine(tf_idf_query,vector)
        docs_to_check[doc] = doc_sim
    
    return(docs_to_check)
        
    
        
    

In [129]:
search_engine2(df)

Search these words:  horrific man sword


{0: 0.5773502691896258,
 99: 0.7774061826549155,
 152: 0.5773502691896257,
 318: 0.9668636346553368,
 504: 0.5773502691896257,
 664: 0.5773502691896257,
 689: 0.5773502691896257,
 1686: 0.5773502691896257,
 2238: 0.9668636346553365,
 2614: 0.5773502691896258,
 2720: 0.5773502691896257,
 2754: 0.5773502691896257,
 3027: 0.5773502691896256,
 3327: 0.5773502691896257,
 3348: 0.5773502691896257,
 4375: 0.5773502691896257,
 7253: 0.5773502691896258,
 11448: 0.5773502691896257,
 14226: 0.5773502691896257,
 17443: 0.5773502691896257,
 17658: 0.5773502691896258,
 14: 0.5773502691896257,
 16: 0.5773502691896257,
 32: 0.5773502691896258,
 39: 0.5773502691896257,
 41: 0.5773502691896258,
 42: 0.5773502691896257,
 63: 0.5773502691896257,
 69: 0.5773502691896257,
 73: 0.5773502691896257,
 74: 0.5773502691896257,
 82: 0.5773502691896257,
 95: 0.5773502691896257,
 98: 0.5773502691896258,
 104: 0.5773502691896257,
 108: 0.5773502691896257,
 135: 0.5773502691896257,
 143: 0.5773502691896258,
 153: 0.57

# 3. Define a new score

In [31]:

def search_engine3(dataset,k):
    # inputs
    query = str(input("Search these words: "))
    type_ = str(input('Movie, Music, ONA, OVA, Special, TV or nan '))
    if type_ not in ['Movie', 'Music', 'ONA', 'OVA', 'Special', 'TV', 'nan','']:
        raise ValueError('Choose between the suggested types')
    try:
        score = float(input("Minimum score between 1 and 10 ") or 0)
    except ValueError:
        raise ValueError('Please input a number')
    query = query.split(" ")

    # getting indices 
    query_indices = []
    for index in query:
        query_indices.append(word_index[index])  
    query_indices = set.intersection(*map(set,listoflists(query_indices)))
    
    if type_ != '':
        type_indices = set(dataset.index[dataset.animeType == type_])
    score_indices = set(dataset.index[dataset.animeScore >= score])
    
    # using Jaccard similarity to get best results 
    output_indices = {}
    for index in tqdm(query_indices):
        vect = [0,0,0]
        if index in query_indices:
            vect[0] = 1
        if index in type_indices:
            vect[1] = 1
        if index in score_indices:
            vect[2] = 1
        js = round(jaccard_score(y_true=[1,1,1],y_pred=vect),2)
        if js in output_indices:
            output_indices[js].append(index)
        else:
            output_indices[js] = [index]
    output_indices = collections.OrderedDict(sorted(output_indices.items(),reverse=True))
    res=[]
    for listy in output_indices.values():
        for el in listy:
            res.append(el)
            if len(res) == k:
                break
        if len(res) == k:
            break
    
    
    return(displayresults(res))

In [42]:
search_engine3(dataset=df,k = 10)

Search these words:  saiyan
Movie, Music, ONA, OVA, Special, TV or nan  TV
Minimum score between 1 and 10  4


100%|████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 1111.46it/s]


,Anime Title,Anime Description,Anime URL
0,Dragon Ball Super,"Seven years after the events of Dragon Ball Z,...",https://myanimelist.net/anime/25183/Gangsta\n
1,Dragon Ball Z,Five years after winning the World Martial Art...,https://myanimelist.net/anime/813/Dragon_Ball_Z\n
2,Dragon Ball Z Movie 10: Kiken na Futari! Super...,"After his loss to Goku, Broly crash lands and ...",https://myanimelist.net/anime/11809/gdgd_Fairi...
3,Dragon Ball Z: The Real 4-D at Super Tenkaichi...,Dragon Ball Z: The Real 4-D at Super Tenkaichi...,https://myanimelist.net/anime/30278/Ghost_Mess...
4,Dragon Ball Z Movie 11: Super Senshi Gekiha!! ...,"Jaga Bada, Mr. Satan's old sparring partner, h...",https://myanimelist.net/anime/9172/Kaitei_Sanm...
5,Dragon Ball Super: Broly,"Forty-one years ago on Planet Vegeta, home of ...",https://myanimelist.net/anime/36946/Dragon_Bal...
6,Dragon Ball: Ossu! Kaettekita Son Gokuu to Nak...,Based on an original concept by the original a...,https://myanimelist.net/anime/12225/Galaxy_Ang...
7,"Dragon Ball Z Movie 08: Moetsukiro!! Nessen, R...",As Goku investigates the destruction of the So...,https://myanimelist.net/anime/5684/Fresh_Precu...
8,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...
9,Dragon Ball Z Movie 14: Kami to Kami,"Following the defeat of a great adversary, Gok...",https://myanimelist.net/anime/36824/Huyao_Xiao...
